In [1]:
import zipfile
import pandas as pd
import os
from datetime import datetime, timedelta
import requests
import time

# getCAISOLMP Function

In [2]:
def getCAISOLMP(startdate="20230101", enddate="20230104", query_name = "PRC_LMP", market_run_id="DAM",nodes=["ALL"],version="12"):
    '''
    OASIS URL request examples: https://developer.caiso.com/Artifacts/OASIS/TechSpecs/OASIS%20API%20URL%207.3.0%20Examples.pdf
    Equivalent to OASIS > Prices > Energy Prices > Location Marginal Prices (LMP)
    Equivalent to OASIS > Prices > Energy Prices > Interval Location Marginal Prices (LMP)
    Parameters:
    startdate: string - beginning date
    enddate: string - end date (exclusive)
    query_name: string
        options are "PRC_LMP" and "PRC_INTVL_LMP"        
    market_run_id: string
        options are "DAM" and "RSUC" for LMP
        options are "RTM" for INTVL_LMP
    nodes: string list
        options:
            list of node strings and ["ALL"]
    
    '''
    start = datetime.strptime(startdate,"%Y%m%d")
    end = datetime.strptime(enddate,"%Y%m%d")
  # Initialize data frame with starting day
    activeDay = start
  
  #define base URL
    baseURL = "http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6" #resultformat=6
    if(nodes!=["ALL"]):
        print(nodes)
        data = pd.DataFrame()
        while(activeDay < end):
            nextDay = activeDay +timedelta(days=1)
            j = 300
            for node in nodes:
                j +=1
                activeNode = node  
          # assemble url for LMP
                getURL = baseURL + "&queryname=" + query_name + "&startdatetime=" +  activeDay.strftime("%Y%m%d") + "T08:00-0000" + "&enddatetime=" + nextDay.strftime("%Y%m%d") + "T08:00-0000"+ "&version=" + version + "&market_run_id=" + market_run_id+ "&node=" + activeNode
                #print(getURL)
                r = requests.get(getURL)  
                new_file_name = str(j) + '.zip'
                time.sleep(5)
                with open(new_file_name, 'wb') as f:
                    f.write(r.content)
                partial_data = pd.read_csv(new_file_name)
                data = data.append(partial_data, ignore_index=True)
      
            activeDay = activeDay + timedelta(days=1) # add one day 
    else: #...or get all of the nodes...
    
         #get all nodes day by day.  
        print("all at once")
      # First day -- initialize data output frame.
        activeDay = activeDay + timedelta(days=1)
        getURL = baseURL + "&queryname=" + query_name +"&startdatetime=" + start.strftime("%Y%m%d") + "T08:00-0000"+ "&enddatetime=" + activeDay.strftime("%Y%m%d") + "T00:00-0000"+ "&version=" + version + "&market_run_id="+market_run_id +"&grp_type=ALL_APNODES"
        r = requests.get(getURL)  
        with open('zip_folder.zip', 'wb') as f:
            f.write(r.content)
        data = pd.read_csv('zip_folder.zip')# unzip and read csv, dump into data
        f.close()
        os.remove('zip_folder.zip')
        k = 1
        while(activeDay < end):
            time.sleep(5)
            nextDay = activeDay +timedelta(days=1)
            getURL = "http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=" + query_name+"&startdatetime=" + activeDay.strftime("%Y%m%d") + "T08:00-0000"+ "&enddatetime=" + nextDay.strftime("%Y%m%d") + "T00:00-0000"+ "&version=" + version + "&market_run_id="+market_run_id +"&grp_type=ALL_APNODES"
            r = requests.get(getURL)  
            new_file_name = str(k) + '.zip'
            print(new_file_name)
            with open(new_file_name, 'wb') as f:
                f.write(r.content)
            partial_data = pd.read_csv(new_file_name)# unzip and read csv, dump into data
            data = data.append(partial_data)
            f.close()
            activeDay = activeDay + timedelta(days=1) #increment one day
            k += 1
               
    return data

## Usage Example 1
Runs the function with all default parameters. Makes a request Prices LMP from Jan 1, 2023 through Jan 3, 2023 with DAM market id for all nodes all at once.

In [3]:
df1 = getCAISOLMP() # run with all default params
df1

all at once
1.zip
2.zip


,INTERVALSTARTTIME_GMT,INTERVALENDTIME_GMT,OPR_DT,OPR_HR,OPR_INTERVAL,NODE_ID_XML,NODE_ID,NODE,MARKET_RUN_ID,LMP_TYPE,XML_DATA_ITEM,PNODE_RESMRID,GRP_TYPE,POS,MW,GROUP
0,2023-01-02T06:00:00-00:00,2023-01-02T07:00:00-00:00,2023-01-01,23,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,140.33040,1
1,2023-01-01T23:00:00-00:00,2023-01-02T00:00:00-00:00,2023-01-01,16,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,86.59849,1
2,2023-01-01T22:00:00-00:00,2023-01-01T23:00:00-00:00,2023-01-01,15,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,56.54871,1
3,2023-01-02T07:00:00-00:00,2023-01-02T08:00:00-00:00,2023-01-01,24,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,130.25111,1
4,2023-01-01T13:00:00-00:00,2023-01-01T14:00:00-00:00,2023-01-01,6,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,110.19324,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267595,2023-01-03T17:00:00-00:00,2023-01-03T18:00:00-00:00,2023-01-03,10,0,YU_6_TGEN-APND,YU_6_TGEN-APND,YU_6_TGEN-APND,DAM,MGHG,LMP_GHG_PRC,YU_6_TGEN-APND,ALL_APNODES,0,0.00000,11165
267596,2023-01-04T05:00:00-00:00,2023-01-04T06:00:00-00:00,2023-01-03,22,0,YU_6_TGEN-APND,YU_6_TGEN-APND,YU_6_TGEN-APND,DAM,MGHG,LMP_GHG_PRC,YU_6_TGEN-APND,ALL_APNODES,0,0.00000,11165
267597,2023-01-03T21:00:00-00:00,2023-01-03T22:00:00-00:00,2023-01-03,14,0,YU_6_TGEN-APND,YU_6_TGEN-APND,YU_6_TGEN-APND,DAM,MGHG,LMP_GHG_PRC,YU_6_TGEN-APND,ALL_APNODES,0,0.00000,11165
267598,2023-01-03T10:00:00-00:00,2023-01-03T11:00:00-00:00,2023-01-03,3,0,YU_6_TGEN-APND,YU_6_TGEN-APND,YU_6_TGEN-APND,DAM,MGHG,LMP_GHG_PRC,YU_6_TGEN-APND,ALL_APNODES,0,0.00000,11165


In [4]:
# filtering the dataframe to make it easier (only one node and LMP type sorted by time) to see the results
df1[(df1["NODE"]=="POD_GARNET_1_WIND-APND") & (df1["LMP_TYPE"]=="LMP")].sort_values(by=['INTERVALSTARTTIME_GMT'])

,INTERVALSTARTTIME_GMT,INTERVALENDTIME_GMT,OPR_DT,OPR_HR,OPR_INTERVAL,NODE_ID_XML,NODE_ID,NODE,MARKET_RUN_ID,LMP_TYPE,XML_DATA_ITEM,PNODE_RESMRID,GRP_TYPE,POS,MW,GROUP
153621,2023-01-01T08:00:00-00:00,2023-01-01T09:00:00-00:00,2023-01-01,1,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,119.50237,6401
153612,2023-01-01T09:00:00-00:00,2023-01-01T10:00:00-00:00,2023-01-01,2,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,114.00266,6401
153617,2023-01-01T10:00:00-00:00,2023-01-01T11:00:00-00:00,2023-01-01,3,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,113.33344,6401
153618,2023-01-01T11:00:00-00:00,2023-01-01T12:00:00-00:00,2023-01-01,4,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,108.80537,6401
153606,2023-01-01T12:00:00-00:00,2023-01-01T13:00:00-00:00,2023-01-01,5,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,107.75499,6401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153622,2023-01-04T03:00:00-00:00,2023-01-04T04:00:00-00:00,2023-01-03,20,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,203.86993,6401
153604,2023-01-04T04:00:00-00:00,2023-01-04T05:00:00-00:00,2023-01-03,21,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,203.80423,6401
153608,2023-01-04T05:00:00-00:00,2023-01-04T06:00:00-00:00,2023-01-03,22,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,195.40220,6401
153618,2023-01-04T06:00:00-00:00,2023-01-04T07:00:00-00:00,2023-01-03,23,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,183.90588,6401


## Usage Example 2
Runs with a list of nodes instead of all at once.

In [5]:
df2 = getCAISOLMP(startdate="20230101", enddate="20230104", query_name = "PRC_LMP", market_run_id="DAM",nodes=["AFPR_1_TOT_GEN-APND", "POD_GARNET_1_WIND-APND"], version="12")
df2

['AFPR_1_TOT_GEN-APND', 'POD_GARNET_1_WIND-APND']
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&startdatetime=20230101T08:00-0000&enddatetime=20230102T08:00-0000&version=12&market_run_id=DAM&node=AFPR_1_TOT_GEN-APND
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&startdatetime=20230101T08:00-0000&enddatetime=20230102T08:00-0000&version=12&market_run_id=DAM&node=POD_GARNET_1_WIND-APND
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&startdatetime=20230102T08:00-0000&enddatetime=20230103T08:00-0000&version=12&market_run_id=DAM&node=AFPR_1_TOT_GEN-APND
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&startdatetime=20230102T08:00-0000&enddatetime=20230103T08:00-0000&version=12&market_run_id=DAM&node=POD_GARNET_1_WIND-APND
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&startdatetime=20230103T08:00-0000&enddatetime=20230104T08:00-0000&version=12&market_run_i

,INTERVALSTARTTIME_GMT,INTERVALENDTIME_GMT,OPR_DT,OPR_HR,OPR_INTERVAL,NODE_ID_XML,NODE_ID,NODE,MARKET_RUN_ID,LMP_TYPE,XML_DATA_ITEM,PNODE_RESMRID,GRP_TYPE,POS,MW,GROUP
0,2023-01-02T02:00:00-00:00,2023-01-02T03:00:00-00:00,2023-01-01,19,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,146.71380,1
1,2023-01-01T08:00:00-00:00,2023-01-01T09:00:00-00:00,2023-01-01,1,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,118.47953,1
2,2023-01-01T16:00:00-00:00,2023-01-01T17:00:00-00:00,2023-01-01,9,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,102.23182,1
3,2023-01-01T21:00:00-00:00,2023-01-01T22:00:00-00:00,2023-01-01,14,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,47.44513,1
4,2023-01-01T09:00:00-00:00,2023-01-01T10:00:00-00:00,2023-01-01,2,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,113.14323,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2023-01-04T03:00:00-00:00,2023-01-04T04:00:00-00:00,2023-01-03,20,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,MGHG,LMP_GHG_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,0.00000,5
716,2023-01-04T05:00:00-00:00,2023-01-04T06:00:00-00:00,2023-01-03,22,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,MGHG,LMP_GHG_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,0.00000,5
717,2023-01-03T10:00:00-00:00,2023-01-03T11:00:00-00:00,2023-01-03,3,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,MGHG,LMP_GHG_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,0.00000,5
718,2023-01-03T11:00:00-00:00,2023-01-03T12:00:00-00:00,2023-01-03,4,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,MGHG,LMP_GHG_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,0.00000,5


In [6]:
# filtering the dataframe to make it easier (LMP type sorted by time) to see the results
df2[(df2["LMP_TYPE"]=="LMP")].sort_values(by=['INTERVALSTARTTIME_GMT'])

,INTERVALSTARTTIME_GMT,INTERVALENDTIME_GMT,OPR_DT,OPR_HR,OPR_INTERVAL,NODE_ID_XML,NODE_ID,NODE,MARKET_RUN_ID,LMP_TYPE,XML_DATA_ITEM,PNODE_RESMRID,GRP_TYPE,POS,MW,GROUP
1,2023-01-01T08:00:00-00:00,2023-01-01T09:00:00-00:00,2023-01-01,1,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,118.47953,1
131,2023-01-01T08:00:00-00:00,2023-01-01T09:00:00-00:00,2023-01-01,1,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,119.50237,1
126,2023-01-01T09:00:00-00:00,2023-01-01T10:00:00-00:00,2023-01-01,2,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,114.00266,1
4,2023-01-01T09:00:00-00:00,2023-01-01T10:00:00-00:00,2023-01-01,2,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,113.14323,1
6,2023-01-01T10:00:00-00:00,2023-01-01T11:00:00-00:00,2023-01-01,3,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,112.34070,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,2023-01-04T05:00:00-00:00,2023-01-04T06:00:00-00:00,2023-01-03,22,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,185.34460,1
605,2023-01-04T06:00:00-00:00,2023-01-04T07:00:00-00:00,2023-01-03,23,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,183.90588,1
486,2023-01-04T06:00:00-00:00,2023-01-04T07:00:00-00:00,2023-01-03,23,0,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,DAM,LMP,LMP_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,176.50410,1
619,2023-01-04T07:00:00-00:00,2023-01-04T08:00:00-00:00,2023-01-03,24,0,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,DAM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,165.43605,1


### Export data to a csv for Don Jacob's market analysis notebooks or analysis in Excel

In [7]:
df2.to_csv("test_final.csv") # code to turn the dataframe into a CSV

## Usage Example 3
Interval LMP data with list of two nodes.

In [8]:
df3 = getCAISOLMP(startdate="20230101", enddate="20230103", query_name = "PRC_INTVL_LMP", market_run_id="RTM", nodes=["AFPR_1_TOT_GEN-APND", "POD_GARNET_1_WIND-APND"], version="3")
df3

['AFPR_1_TOT_GEN-APND', 'POD_GARNET_1_WIND-APND']
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_INTVL_LMP&startdatetime=20230101T08:00-0000&enddatetime=20230102T08:00-0000&version=3&market_run_id=RTM&node=AFPR_1_TOT_GEN-APND
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_INTVL_LMP&startdatetime=20230101T08:00-0000&enddatetime=20230102T08:00-0000&version=3&market_run_id=RTM&node=POD_GARNET_1_WIND-APND
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_INTVL_LMP&startdatetime=20230102T08:00-0000&enddatetime=20230103T08:00-0000&version=3&market_run_id=RTM&node=AFPR_1_TOT_GEN-APND
http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_INTVL_LMP&startdatetime=20230102T08:00-0000&enddatetime=20230103T08:00-0000&version=3&market_run_id=RTM&node=POD_GARNET_1_WIND-APND


,INTERVALSTARTTIME_GMT,INTERVALENDTIME_GMT,OPR_DT,OPR_HR,NODE_ID_XML,NODE_ID,NODE,MARKET_RUN_ID,LMP_TYPE,XML_DATA_ITEM,PNODE_RESMRID,GRP_TYPE,POS,VALUE,OPR_INTERVAL,GROUP
0,2023-01-01T08:00:00-00:00,2023-01-01T08:05:00-00:00,2023-01-01,1,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,RTM,MCC,LMP_CONG_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,0.0000,1,1
1,2023-01-01T08:05:00-00:00,2023-01-01T08:10:00-00:00,2023-01-01,1,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,RTM,MCC,LMP_CONG_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,0.0000,2,1
2,2023-01-01T08:10:00-00:00,2023-01-01T08:15:00-00:00,2023-01-01,1,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,RTM,MCC,LMP_CONG_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,0.0000,3,1
3,2023-01-01T08:15:00-00:00,2023-01-01T08:20:00-00:00,2023-01-01,1,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,RTM,MCC,LMP_CONG_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,0.0000,4,1
4,2023-01-01T08:20:00-00:00,2023-01-01T08:25:00-00:00,2023-01-01,1,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,RTM,MCC,LMP_CONG_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,0.0024,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,2023-01-03T07:35:00-00:00,2023-01-03T07:40:00-00:00,2023-01-02,24,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,RTM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,149.9988,8,5
5756,2023-01-03T07:40:00-00:00,2023-01-03T07:45:00-00:00,2023-01-02,24,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,RTM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,149.9968,9,5
5757,2023-01-03T07:45:00-00:00,2023-01-03T07:50:00-00:00,2023-01-02,24,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,RTM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,148.3800,10,5
5758,2023-01-03T07:50:00-00:00,2023-01-03T07:55:00-00:00,2023-01-02,24,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,POD_GARNET_1_WIND-APND,RTM,LMP,LMP_PRC,POD_GARNET_1_WIND-APND,ALL_APNODES,0,146.7492,11,5
